In [18]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../application/.env'))

True

In [19]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector
from langchain.schema.messages import get_buffer_string
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import format_document
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import ChatPromptTemplate

host = os.getenv("PG_VECTOR_HOST")
user = os.getenv("PG_VECTOR_USER")
password = os.getenv("PG_VECTOR_PASSWORD")
COLLECTION_NAME = os.getenv("PGDATABASE")
CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:5432/{COLLECTION_NAME}"

embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)
retriever = store.as_retriever()

model = ChatOpenAI()

In [20]:
retriever.get_relevant_documents("When are the opening hours?")

[Document(page_content='Monday to Thursday: 11:00 AM - 11:00 PM Friday: 11:00 AM - 12:00 AM (midnight) Saturday: 10:00 AM - 12:00 AM (midnight) Sunday: 10:00 AM - 11:00 PM Special Hours: Our kitchen closes 30 minutes before', metadata={'source': 'opening_hours.txt'}),
 Document(page_content='the exact amount before confirming your order. Restaurant Opening Hours:', metadata={'source': 'opening_hours.txt'}),
 Document(page_content='La Tavola Calda - Delivery Service & Opening Hours', metadata={'source': 'opening_hours.txt'}),
 Document(page_content="30 minutes before the restaurant closing time. Whether you're craving a quick lunch, planning a cozy dinner at home, or simply indulging in a late-night snack, La Tavola Calda is just a chat away.", metadata={'source': 'opening_hours.txt'})]

In [21]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


In [22]:
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [23]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)

In [24]:
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

In [25]:
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser()

In [26]:
conversational_qa_chain.invoke(
    {
        "question": "When are the opening hours?",
        "chat_history": [],
    }
)

Intermediate Output: {'standalone_question': 'What are the opening hours?'}


"The restaurant's opening hours are as follows:\nMonday to Thursday: 11:00 AM - 11:00 PM\nFriday: 11:00 AM - 12:00 AM (midnight)\nSaturday: 10:00 AM - 12:00 AM (midnight)\nSunday: 10:00 AM - 11:00 PM"